In [ ]:
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# Read the CSV file
training_data_file = 'C:/Users/myrsi/Documents/msc_thesis/training_data.csv'
validation_data_file = 'C:/Users/myrsi/Documents/msc_thesis/validation_data.csv'
df_training_data = pd.read_csv(training_data_file)
df_validation_data = pd.read_csv(validation_data_file)

# Remove rows with non-zero values in 'Doppler_Vel' column
df_training_data_static = df_training_data[df_training_data['Doppler_Vel'] == 0]
df_validation_data_static = df_validation_data[df_validation_data['Doppler_Vel'] == 0]

# Extract features and labels
X = df_training_data_static[['Angle', 'RCS', 'Range_Sc']].values
y = df_training_data_static['Label_ID'].values
X_val = df_validation_data_static[['Angle', 'RCS', 'Range_Sc']].values
y_val = df_validation_data_static['Label_ID'].values

# Split the training data into training and test sets
val_proportion = df_validation_data_static.shape[0] / df_training_data_static.shape[0]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=val_proportion,random_state=42)

# Convert data to tensors
X_train = torch.Tensor(X_train)
y_train = torch.Tensor(y_train)
X_test = torch.Tensor(X_test)
y_test = torch.Tensor(y_test)
X_val = torch.Tensor(X_val)
y_val = torch.Tensor(y_val)

# Create TensorDatasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
val_dataset = TensorDataset(X_val, y_val)

# Create DataLoaders
batch_size = 5
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#Grid map

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# Define the model architecture
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=6)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=9)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(2,2)
        
        self.upsample1 = nn.Upsample(size=2)
        self.deconv1 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=9)
        self.bn5 = nn.BatchNorm2d(128)
        self.deconv2 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=6)
        self.bn6 = nn.BatchNorm2d(64)
        self.upsample2 = nn.Upsample(size=2)
        self.deconv3 = nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=5)
        self.bn7 = nn.BatchNorm2d(128)
        self.deconv4 = nn.ConvTranspose2d(in_channels=32, out_channels=6, kernel_size=3)

        
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x))) 
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)    
        x = F.relu(self.bn3(self.conv3(x))) 
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)     

        x = self.upsample1(x)
        x = F.relu(self.bn5(self.deconv1(x))) 
        x = F.relu(self.bn6(self.deconv2(x)))
        x = self.upsample2(x)
        x = F.relu(self.bn7(self.deconv3(x))) 
        x = self.deconv4(x)


        x = x.view(-1, 64 * 8 * 8)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.softmax(self.fc2(x))

        return x



#instantiate a neural network model

model = Net().to(device)
print(model)

In [ ]:
# Define the loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9,0.999))

In [ ]:
torch.manual_seed(42)

epochs = 5

# Put data to target device
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)
X_val, y_val = X_val.to(device), y_val.to(device)


for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        X_train, y_train = data

        # forward pass
        y_pred = model(X_train)
        
        # Calculate loss
        loss = loss_fn(y_pred, y_train)
        
        # Optimizer zero grad
        optimizer.zero_grad()
        
        # Loss backward 
        loss.backward
        
        # Optimizer step
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')